# Permuted Pixel MNIST Demo

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
data_dir = "./MNIST_data"
mnist = input_data.read_data_sets(data_dir, one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# configurations
learning_rate = 0.001
batch_size =256
training_iters = 30000*batch_size
display_step = 100
testing_step = 1000
n_input = 1
n_steps = 28*28 #timestep
n_hidden = 128#hidden layers
n_classes =10

In [4]:
# permutation seed 
seed = 92916
if 'seed' in globals():
    rng_permute = np.random.RandomState(seed)
    idx_permute = rng_permute.permutation(n_steps)
else:
    idx_permute = np.random.permutation(n_steps)
# shape
mnist.train.images.shape
mnist.train.labels.shape

(55000, 10)

In [5]:
x=tf.placeholder (tf.float32,[None,n_steps,n_input],name='X')
y=tf.placeholder (tf.float32,[None,n_classes],name='Y')
weights ={'out':tf.Variable(tf.random_normal([n_hidden,n_classes]),name='Weight')}
biases ={'out':tf.Variable(tf.random_normal([n_classes]),name='bias')}

In [6]:
# define RNN
def RNN(x,weights,bias):
    x=tf.unstack(x,n_steps,1)
    lstm_cell=rnn.BasicLSTMCell(n_hidden,forget_bias=0.8)
    lstm_cell=rnn.DropoutWrapper(lstm_cell)
    outputs, states =rnn.static_rnn(lstm_cell,x,dtype=tf.float32)
    return tf.matmul(outputs[-1],weights['out']+biases['out'])

In [7]:
pred = RNN(x,weights,biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y),name=None)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('./Summary',sess.graph)
    step = 1
    while step*batch_size < training_iters:
        batch_x,batch_y =mnist.train.next_batch(batch_size)
        batch_x = batch_x[:, idx_permute]
        batch_x = batch_x.reshape(batch_size,n_steps,n_input)
        sess.run(optimizer,feed_dict={x:batch_x,y:batch_y})
        if step % display_step ==0:
            acc = sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            loss = sess.run(cost,feed_dict={x:batch_x,y:batch_y})
            print('Iter:'+str(step)+', Minibatch Loss='+\
             '{:.6f}'.format(loss)+', Trainging Accurary='+\
             '{:.5f}'.format(acc))
        if step % testing_step == 0:
        
            # validation performance
            batch_x,batch_y =mnist.validation.next_batch(batch_size)
            batch_x = batch_x[:, idx_permute]
            batch_x = batch_x.reshape(batch_size,n_steps,n_input)
            acc__ = sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            
            # test performance
            batch_x,batch_y =mnist.test.next_batch(batch_size)
            batch_x = batch_x[:, idx_permute]
            batch_x = batch_x.reshape(batch_size,n_steps,n_input)
            acc_ = sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            
            print ("========> Validation Accuarcy: " + "{:.6f}".format(acc__) \
            + ", Testing Accuarcy: " + "{:.6f}".format(acc_)) 
            print("\n")
        
        step+=1
    print('Optimization Finished!')
    writer.close()
    test_len =128
    test_data = mnist.test.images[:test_len].reshape((-1,n_steps,n_input))
    test_label = mnist.test.labels[:test_len]
    print('========> Final Test Accuracy:', \
          sess.run(accuracy,feed_dict={x:test_data,y:test_label}))

Iter:100, Minibatch Loss=1.879636, Trainging Accurary=0.40625
Iter:200, Minibatch Loss=1.566477, Trainging Accurary=0.46484
Iter:300, Minibatch Loss=1.440592, Trainging Accurary=0.49219
Iter:400, Minibatch Loss=1.423920, Trainging Accurary=0.50781
Iter:500, Minibatch Loss=1.207187, Trainging Accurary=0.55469
Iter:600, Minibatch Loss=1.074005, Trainging Accurary=0.66797
Iter:700, Minibatch Loss=0.953170, Trainging Accurary=0.69531
Iter:800, Minibatch Loss=1.024905, Trainging Accurary=0.69531
Iter:900, Minibatch Loss=0.899064, Trainging Accurary=0.69531
Iter:1000, Minibatch Loss=0.796993, Trainging Accurary=0.73438
========> Validation Accuarcy: 0.738281, Testing Accuarcy: 0.730469


Iter:1100, Minibatch Loss=0.737772, Trainging Accurary=0.74219
Iter:1200, Minibatch Loss=0.742339, Trainging Accurary=0.75781
Iter:1300, Minibatch Loss=0.704195, Trainging Accurary=0.76953
Iter:1400, Minibatch Loss=0.797363, Trainging Accurary=0.71094
Iter:1500, Minibatch Loss=0.735959, Trainging Accurary=0.